## LLM zoomcamp Homework 1

### Q 1 

Run Elastic Search 8.17.6, and get the cluster information. What's the version.build_hash value?


```bash
docker stop $(docker ps -q)
docker rm $(docker ps -aq)

docker pull docker.elastic.co/elasticsearch/elasticsearch:8.17.6

docker run -p 9200:9200 -e "discovery.type=single-node" -e "ES_JAVA_OPTS=-Xms1g -Xmx1g" --memory="2g" -e "xpack.security.enabled=false" docker.elastic.co/elasticsearch/elasticsearch:8.17.6

In [1]:
%pip install 'elasticsearch>=8.0.0,<9.0.0'

  Obtaining dependency information for elasticsearch<9.0.0,>=8.0.0 from https://files.pythonhosted.org/packages/33/62/f62e8a5c7c6f7b27481c9ffc248fb32078ad88878aa4f3731a83a14cc797/elasticsearch-8.18.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.3/906.3 kB 56.7 MB/s eta 0:00:00
  Attempting uninstall: elasticsearch
    Found existing installation: elasticsearch 9.0.2
    Uninstalling elasticsearch-9.0.2:
      Successfully uninstalled elasticsearch-9.0.2

[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from elasticsearch import Elasticsearch
import os 
from dotenv import load_dotenv
import requests

In [3]:
response = requests.get("http://localhost:9200")

In [4]:
data = response.json()

In [5]:
print(f"version.build_hash value is {data["version"]["build_hash"]} ")

version.build_hash value is dbcbbbd0bc4924cfeb28929dc05d82d662c527b7 


In [6]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

### Q 2

Index the data in the same way as was shown in the course videos. Make the course field a keyword and the rest should be text.

Which function do you use for adding your data to elastic?

In [7]:
import minsearch

/workspaces/llm-zoomcamp/week-1/minsearch.py:10: UserWarning: Now minsearch is installable via pip: 'pip install minsearch'. Remove the downloaded file and re-install it with pip.
  warnings.warn(


In [8]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch("http://localhost:9200")
index_name = "llm-zoomcamp"  # Use a valid, lowercase name

mapping = {
    "mappings": {
        "properties": {
            "course": {"type": "keyword"},
            "question": {"type": "text"},
            "text": {"type": "text"},
            "section": {"type": "text"}
        }
    }
}

try:
    es_client.indices.create(index=index_name, body=mapping)
except Exception as e:
    print(e)

# Delete the index if it already exists (optional, for a clean start)
#if es_client.indices.exists(index=index_name):
#    es_client.indices.delete(index=index_name)

# Create the index with the mapping
#es_client.indices.create(index=index_name, body=mapping)

In [9]:
from tqdm.auto import tqdm

/workspaces/llm-zoomcamp/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:07<00:00, 123.29it/s]


In [11]:
es_client = Elasticsearch('http://localhost:9200') 

In [12]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:03<00:00, 299.85it/s]


In [14]:
print(f"We use es_client.index to add data to elastic.")

We use es_client.index to add data to elastic.


### Q3. Searching
Now let's search in our index.

We will execute a query "How do execute a command on a Kubernetes pod?".

Use only question and text fields and give question a boost of 4, and use "type": "best_fields".

What's the score for the top ranking result?

In [15]:
query_0 = "How do execute a command on a Kubernetes pod?"

query = {
    "query": {
        "multi_match": {
            "query": query_0,
            "fields": ["question^4", "text"],
            "type": "best_fields"
        }
    }
}

response = es_client.search(index=index_name, body=query)
top_hits = response['hits']['hits'][0]["_score"]

print(f"Top hits score for query '{query_0}': {top_hits}")


Top hits score for query 'How do execute a command on a Kubernetes pod?': 44.55304


### Q4. Filtering
Now ask a different question: "How do copy a file to a Docker container?".

This time we are only interested in questions from machine-learning-zoomcamp.

Return 3 results. What's the 3rd question returned by the search engine?

In [42]:
query_0 = "How do copy a file to a Docker container?"

query = {
    "size": 3, 
    "query": {
        "bool": {
            "must": [ 
                {
            "multi_match": {
                "query": query_0,
                "fields": ["question", "text"],
                "type": "best_fields"
        }
    }
    ], 
    "filter": [ { "term": {
                "course": "machine-learning-zoomcamp"
            }
        }
    ]
    }
} 
}

response = es_client.search(index=index_name, body=query)
third_question = response["hits"]["hits"][2]["_source"]["question"]
print(f"3rd question: {third_question}")

3rd question: How do I copy files from a different folder into docker container’s working directory?


### Q5. Building a prompt
Now we're ready to build a prompt to send to an LLM.

Take the records returned from Elasticsearch in Q4 and use this template to build the context. Separate context entries by two linebreaks (\n\n)

context_template = """
Q: {question}
A: {text}
""".strip()


In [49]:
context_template = """
Q: {question}
A: {text}
""".strip()

def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = "\n\n".join([
        context_template.format(
            question= doc['_source']['question'],
            text= doc['_source']['text']
        ) for doc in search_results
        ] )
    
    #context = ""
    
    #for doc in search_results:
        
    #    context = context + f"section: {doc['_source']['section']}\nquestion: {doc['_source']['question']}\nanswer: {doc['_source']['text']}\n\n"
    
    #prompt = context_template.format(question=query, context=context).strip()
    
    prompt = prompt_template.format(question = query, context=context)

    return prompt

In [29]:
response["hits"]["hits"]

[{'_index': 'llm-zoomcamp',
  '_id': 'hjfnb5cBqjbfUwpbgod1',
  '_score': 22.95559,
  '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I copy files from my local machine to docker container?',
   'course': 'machine-learning-zoomcamp'}},
 {'_index': 'llm-zoomcamp',
  '_id': '0jfnb5cBqjbfUwpbcIO_',
  '_score': 22.95559,
  '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command

In [50]:
prompt = build_prompt(query_0, response["hits"]["hits"])

In [53]:
print(f"The lenght of the promåpt is { len(prompt)}")

The lenght of the promåpt is 1524
